# Pull Alpaca API data for ARKK holdings

In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from datetime import datetime, timedelta

In [2]:
load_dotenv()

True

In [3]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [4]:
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [5]:
print(alpaca)

## ARK ETF - Alpaca Data Pull

In [6]:
ARK_ETF = ["TSLA", "ROKU", "TDOC", "MSTR", "SQ", "SHOP", "TTWO", "TWLO", "SPOT", "NTLA", "CRSP", "EXAS", "Z", "TWTR", "TER"]

In [7]:
timeframe = "1D"

In [8]:
start_date = pd.Timestamp("2018-09-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-08-01", tz="America/New_York").isoformat()

In [9]:
 ## This code artificially creates 3 years -- could replace code in line 8 --

In [10]:
# today = datetime.now()
# startdate = today - timedelta(days=3*365)

In [11]:
# this is to convert the datetime format to the pd.Timestamp that alpaca api syntax requirement

In [12]:
# end_date = pd.Timestamp(today.strftime("%Y-%m-%d"), tz="America/New_York").isoformat()
# start_date = pd.Timestamp(startdate.strftime("%Y-%m-%d"), tz="America/New_York").isoformat()

In [13]:
ARK_ETF_portfolio = alpaca.get_barset(
    ARK_ETF,
    timeframe,
    start = start_date,
    end = end_date,
    limit=1000
).df

In [14]:
ARK_ETF_portfolio.head()

CRSP                                  EXAS         \
                            open   high     low  close   volume   open   high   
time                                                                            
2018-09-04 00:00:00-04:00  58.50  59.00  53.600  55.50  1032358  74.50  77.75   
2018-09-05 00:00:00-04:00  55.50  55.50  52.005  53.44   606929  79.08  80.35   
2018-09-06 00:00:00-04:00  53.17  53.72  49.580  49.84   558842  76.12  76.92   
2018-09-07 00:00:00-04:00  50.00  51.73  49.360  50.12   431793  74.96  78.95   
2018-09-10 00:00:00-04:00  50.66  50.85  47.500  49.67  1106214  77.51  78.00   

                                                  ...   TWTR                   \
                             low  close   volume  ...   open    high      low   
time                                              ...                           
2018-09-04 00:00:00-04:00  74.26  77.61  1377323  ...  34.75  35.125  34.4800   
2018-09-05 00:00:00-04:00  74.26  75.80  1713741  ...  34.65  34.700  32.5100   
2018-09-06 00:00:00-04:00  74.15  75.04  1039141  ...  32.86  32.950  30.6200   
2018-09-07 00:00:00-04:00  74.41  77.22  1384215  ...  30.31  31.390  29.8200   
2018-09-10 00:00:00-04:00  75.58  75.99   812537  ...  30.50  30.600  29.9531   

                                                Z                        \
                           close    volume   open    high    low  close   
time                                                                      
2018-09-04 00:00:00-04:00  34.84  12200652  48.55  48.900  47.56  47.72   
2018-09-05 00:00:00-04:00  32.73  32198973  47.82  48.270  46.91  47.67   
2018-09-06 00:00:00-04:00  30.82  33189835  47.68  47.980  43.87  44.62   
2018-09-07 00:00:00-04:00  30.49  28893185  44.58  46.115  44.52  44.93   
2018-09-10 00:00:00-04:00  30.54  15451664  45.00  46.200  44.60  46.12   

                                    
                            volume  
time                                
2018-09-04 00:00:00-04:00  1177704  
2018-09-05 00:00:00-04:00  1171127  
2018-09-06 00:00:00-04:00  2537771  
2018-09-07 00:00:00-04:00  1830479  
2018-09-10 00:00:00-04:00  1103775  

[5 rows x 75 columns]

In [15]:
## saving ARK_ETF_portfolio to csv

In [16]:
ARK_ETF_portfolio.to_csv("Ark_ETF.csv")

### Competitors - Alpaca Data Pull

In [17]:
competitors_df = ["NIO", "NFLX", "PFE", "MSFT", "V", "PYPL", "CRM", "ATVI", "VG", "TTD", "IONS", "NVAX", "AMGN", "RDFN", "SNAP"]

In [18]:
timeframe = "1D"

In [19]:
start_date = pd.Timestamp("2018-09-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-08-01", tz="America/New_York").isoformat()

In [20]:
 ## This code artificially creates 3 years -- could replace code in line 16 --

In [21]:
today = datetime.now()
startdate = today - timedelta(days=3*365)

In [22]:
end_date = pd.Timestamp(today.strftime("%Y-%m-%d"), tz="America/New_York").isoformat()
start_date = pd.Timestamp(startdate.strftime("%Y-%m-%d"), tz="America/New_York").isoformat()

In [23]:
competitors_list = alpaca.get_barset(
    competitors_df,
    timeframe,
    start = start_date,
    end = end_date,
    limit=1000
).df

In [24]:
competitors_list.head()

AMGN                                    ATVI  \
                             open    high     low   close   volume   open   
time                                                                        
2018-08-13 00:00:00-04:00  194.69  196.32  193.72  195.60  1404144  70.61   
2018-08-14 00:00:00-04:00  195.94  196.96  194.89  195.78   934342  71.04   
2018-08-15 00:00:00-04:00  194.83  195.48  192.89  195.23  1619580  70.39   
2018-08-16 00:00:00-04:00  194.46  197.13  192.73  196.48  2041279  69.68   
2018-08-17 00:00:00-04:00  196.24  198.51  195.28  197.42  1750815  69.51   

                                                              ...       V  \
                              high     low  close     volume  ...    open   
time                                                          ...           
2018-08-13 00:00:00-04:00  72.3000  70.255  70.52  5205271.0  ...  140.53   
2018-08-14 00:00:00-04:00  71.7359  70.180  71.29  4233814.0  ...  140.30   
2018-08-15 00:00:00-04:00  70.8100  68.445  69.31  7779210.0  ...  140.35   
2018-08-16 00:00:00-04:00  69.9900  68.910  69.69  8359894.0  ...  140.75   
2018-08-17 00:00:00-04:00  69.6700  68.330  69.14  4825534.0  ...  140.78   

                                                                VG         \
                             high      low   close   volume   open   high   
time                                                                        
2018-08-13 00:00:00-04:00  141.24  139.830  140.20  3372702  13.71  13.80   
2018-08-14 00:00:00-04:00  140.99  139.880  140.76  3860391  13.70  14.38   
2018-08-15 00:00:00-04:00  140.69  138.840  139.92  3682605  14.37  14.50   
2018-08-16 00:00:00-04:00  141.49  140.420  140.64  3688784  14.13  14.46   
2018-08-17 00:00:00-04:00  141.71  140.565  141.38  3938667  14.23  14.59   

                                                     
                             low   close     volume  
time                                                 
2018-08-13 00:00:00-04:00  13.66  13.700   671607.0  
2018-08-14 00:00:00-04:00  13.69  14.370  1916199.0  
2018-08-15 00:00:00-04:00  14.09  14.125  1875413.0  
2018-08-16 00:00:00-04:00  14.08  14.240   899309.0  
2018-08-17 00:00:00-04:00  14.14  14.560  1050613.0  

[5 rows x 75 columns]

In [25]:
## saving competitors_portfolio to csv

In [26]:
competitors_list.to_csv("competitors_list.csv")

In [27]:
## Dictionary & weights 

In [28]:
ARK_weight = {
    "TSLA": .1016,
    "ROKU": .0571,
    "TDOC": .0570,
    "MSTR": .010,
    "SQ": .0463,
    "SHOP": .0438,
    "TTWO": .010,
    "TWLO": .0368,
    "SPOT": .0335,
    "NTLA": .0301,
    "CRSP": .0292,
    "EXAS": .0281,
    "Z": .0276,
    "TWTR": .0242,
    "TER": .010, 
}

In [29]:
competitors_weight = {}

In [30]:
for ticker in ARK_weight:
    competitors_weight [competitors_df [ARK_ETF.index(ticker)]] = ARK_weight[ticker]                

In [31]:
print(competitors_weight)

{'NIO': 0.1016, 'NFLX': 0.0571, 'PFE': 0.057, 'MSFT': 0.01, 'V': 0.0463, 'PYPL': 0.0438, 'CRM': 0.01, 'ATVI': 0.0368, 'VG': 0.0335, 'TTD': 0.0301, 'IONS': 0.0292, 'NVAX': 0.0281, 'AMGN': 0.0276, 'RDFN': 0.0242, 'SNAP': 0.01}
